# COGS 108 - Final Project: Vehicle Stops in San Diego

### Group Members: 
- Louise Xu (A12650425)
- David Tay (A10914210)
- Jeffery Lu

### Goal of Project:
- To determine whether there is a correlation between the frequency of vehicle stops in San Diego and the time and locations at which they take place
- To examine possible reasons for the correlations, if any exist 

### Research Question
- Are there more vehicle stops at certain times and places than others? Does the frequency of vehicle stops vary significantly between different seasons?

### Hypothesis
- There will most likely be more traffic violations during periods of time with high traffic density (e.g. rush hour), during weekdays (when there are more people on the road who are driving to work), and during holiday seasons (since there would be more tourists and people visting family). On a broader scale, given that the population of San Diego is increasing, it reasonable to say that the number of traffic violations is also increasing at a similar rate. 
- We can also guess that the number of vehicle stops in more populated areas is greater than those in less populated areas.

In [98]:
# Import packages
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
from scipy.stats import ttest_ind

## Part 1: Load and Clean Data

In [99]:
# Import each vehicle stops data file into its own DataFrame
vs_14 = pd.read_csv('vehicle_stops_2014_datasd.csv')
vs_15 = pd.read_csv('vehicle_stops_2015_datasd.csv')
vs_16 = pd.read_csv('vehicle_stops_2016_datasd.csv')
vs_17 = pd.read_csv('vehicle_stops_2017_datasd.csv')

# Join the data for each year together into one DataFrame
# Note: The file for vs_17 contained two more columns (both of them empty and unnamed) than the datasets for the other years 
# These two columns were deleted on Microsoft Excel in order to successfully merge the four datasets together
vs_all = pd.concat([vs_14, vs_15, vs_16, vs_17], ignore_index=True)

#Import police regions into a DataFrame
sd_neigh = pd.read_csv('pd_beat_neighborhoods_datasd.csv')

In [100]:
# Let's check to see how many observations are in each dataset
# Note: vs_17 is year to date data from 2017, so there is data up to May of 2017 (the period in which this analysis is conducted)
print(vs_14.shape)
print(vs_15.shape)
print(vs_16.shape)
print(vs_17.shape)

(144164, 15)
(115422, 15)
(103051, 15)
(28362, 15)


We can see here that the number of vehicle stops is actually *decreasing* each year between 2014 and 2016. Also, while we can not take the 2017 data into direct consideration because we only have data up to May, we can predict the total number of vehicle stops at the end of this year based on our current data (28362 stops) divided by the five months that have already passed and then multiplied by the twelve months of the year, which results in (28362)(12)/5 = 68068.8 stops. We can round up to the nearest ten thousandsth and estimate that approximately 70000 vehicle stops will happen in the year of 2017. This still follows the trend of decreasing vehicle stops in San Diego.

In [101]:
# Let's start cleaning up the data

# We won't be using these columns
vs_all.drop(['arrested', 'searched', 'obtained_consent', 'contraband_found', 'property_seized'], axis=1, inplace=True, errors='ignore')

# Get rid of rows with invalid data
vs_all.dropna(inplace=True)
vs_all = vs_all.loc[(vs_all['service_area'] != 'Unknown')]

# Only keep data of stops involving San Diego residents
# vs_all = vs_all.loc[(vs_all['sd_resident'] == 'Y')]

# Cleaning the ages of the subjects, these characters occur in the dataset and we don't want these
vs_all = vs_all.loc[~(vs_all['subject_age'].str.contains("_|N|`|Q|F|Y|H", case=False))]
vs_all[['subject_age']] = vs_all[['subject_age']].apply(pd.to_numeric)

# Some stops have an X as the gender
vs_all = vs_all.loc[(vs_all['subject_sex'].str.contains("M|F", case=False))]

# Use Safe Harbor practices on the subject_ages
vs_all = vs_all.loc[(vs_all['subject_age'] < 90) | (vs_all['subject_age'] >= 16)] 

# Look at shape of data so far
vs_all


,stop_id,stop_cause,service_area,subject_race,subject_sex,subject_age,timestamp,stop_date,stop_time,sd_resident
0,1044975,Moving Violation,110,W,M,24.0,2014-01-01 01:25:00,2014-01-01,1:25,Y
1,1044976,Moving Violation,320,W,M,42.0,2014-01-01 05:47:00,2014-01-01,5:47,Y
2,1044977,Moving Violation,320,L,M,29.0,2014-01-01 07:46:00,2014-01-01,7:46,Y
3,1044978,Moving Violation,610,W,M,23.0,2014-01-01 08:10:00,2014-01-01,8:10,Y
4,1044980,Equipment Violation,930,H,M,35.0,2014-01-01 08:35:00,2014-01-01,8:35,N
5,1044979,Equipment Violation,820,H,M,30.0,2014-01-01 08:39:00,2014-01-01,8:39,Y
6,1044981,Moving Violation,710,H,F,19.0,2014-01-01 09:13:00,2014-01-01,9:13,N
7,1045139,Moving Violation,120,W,M,32.0,2014-01-01 09:50:00,2014-01-01,9:50,Y
8,1045141,Moving Violation,120,W,M,36.0,2014-01-01 10:00:00,2014-01-01,10:00,Y
9,1045140,Moving Violation,120,H,M,27.0,2014-01-01 10:40:00,2014-01-01,10:40,Y


## Part 2: Exploratory Data Vizualization

## Part 3: Exploring The Data

## Part 4: Data Analysis

## Part 5: Discussion & Conclusions